In [41]:
import aerospike
import json
from aerospike import predicates
config = {
    'hosts': [
        ( 'bidapi-g-use1d-as-p-c8-1.srv.media.net', 3000 )
    ],
    'policies': {
        'timeout': 10000 # milliseconds
    }
}
client = aerospike.client(config).connect()

: 

In [57]:
import json

results = {}
site_names = {}

def collect_results(record):
    # Assuming the JSON string is in the third element of the tuple
    try:
        json_data = json.loads(record[2][''])
        for item in json_data:
            sd_value = item.get('sd')
            sn_value = item.get('sn')
            print(f"SD: {sd_value}, SN: {sn_value}")
            if sd_value:
                results[sd_value] = results.get(sd_value, 0) + 1
            if sn_value:
                site_names[sn_value] = site_names.get(sn_value, 0) + 1
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error processing record: {e}")

try:
    query = client.query("replicator", "RPM_CUSTOMER_STOP_LOSS")
    query.foreach(collect_results)
    sd_output = json.dumps(results)
    site_output = json.dumps(site_names)
    print(sd_output)
    print(site_output)
    # print(len(results))
    # print(len(site_names))
except Exception as e:
    print(f"Error: {e}")

: 

In [65]:
import matplotlib.pyplot as plt
from datetime import datetime
from collections import defaultdict
# Initialize dictionaries to hold monthly and yearly counts
monthly_counts = defaultdict(int)
yearly_counts = defaultdict(int)
datewise_counts = defaultdict(int)

# Process each date and aggregate counts
for date_str, count in results.items():
    date = datetime.strptime(date_str, "%Y%m%d%H%M%S")
    year_month = date.strftime("%Y-%m")
    year = date.strftime("%Y")
    date_str = date.strftime("%Y-%m-%d-%m")

    monthly_counts[year_month] += count
    yearly_counts[year] += count
    datewise_counts[date_str] += count

# Sort months
sorted_months = dict(sorted(monthly_counts.items()))
sorted_dates = list(sorted_months.keys())
sorted_datewise = dict(sorted(datewise_counts.items()))

# Plotting Monthly Counts
plt.figure(figsize=(12, 6))
bars = plt.bar(sorted_months.keys(), sorted_months.values(), color='skyblue')
plt.title("Monthly Counts")
plt.xlabel("Month")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')

# Add value labels on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# Plotting Yearly Counts
plt.figure(figsize=(8, 6))
bars = plt.bar(yearly_counts.keys(), yearly_counts.values(), color='lightgreen')
plt.title("Yearly Counts")
plt.xlabel("Year")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')

# Add value labels on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# Plotting Datewise Counts
plt.figure(figsize=(14, 6))
bars = plt.bar(sorted_datewise.keys(), sorted_datewise.values(), color='salmon')
plt.title("Datewise Counts")
plt.xlabel("Date")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')

# Add value labels on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

: 